In [1]:
import requests
import json

import pandas as pd
import matplotlib.pyplot as plt

In [14]:
ETHERSCAN_API_TOKEN = ''

In [26]:
class TokenBalanceStorage:

    def __init__(self, api_key: str, user: str):
        self.___api_key: str = api_key
        self.user: str = user
        self.tokens_data: dict = {}

    def _transform_transactions_data(self, json_str: str) -> pd.DataFrame:
        """Transforms json string to pandas DataFrame."""
        res_df = pd.DataFrame(json.loads(json_str)['result'])
        res_df = res_df.astype({
            'blockNumber': 'int64',
            'value': 'int64',
            'tokenDecimal': 'int64',
            'timeStamp': 'int64'
        })
        res_df['balance'] = res_df.value / (10 ** res_df.tokenDecimal)
        res_df = res_df[['blockNumber','from','to','balance','timeStamp']]

        # negative balance means that the transaction is a transfer from the address
        # positive balance means that the transaction is a transfer to the address
        res_df.loc[res_df['from'] == self.user.lower(), 'balance'] = -res_df['balance']

        # sort by blockNumber and timeStamp
        res_df.sort_values(['blockNumber', 'timeStamp'], inplace=True, ascending=False)
        return res_df

    def _get_user_transactions(self, token_address: str, user: str) -> pd.DataFrame:
        """Returns DataFrame with user transactions for specified token and chain."""

        start_block = 0
        flag = True
        total_df = pd.DataFrame()
        
        while flag:
            url = f'https://api.arbiscan.io/api?module=account&action=tokentx&contractaddress={token_address}'+\
            f'&address={user}&page=1&offset=10000&startblock={start_block}&endblock=99999999&sort=asc&apikey={self.___api_key}'
            res = requests.get(url)
            print(res.json())
            res_df = self._transform_transactions_data(res.text)
            total_df = pd.concat([total_df, res_df])
            
            start_block = total_df.blockNumber.iloc[-1] + 1
            
            if len(res_df) < 10**4:
                flag = False
        
        return total_df
    
    def get_token_balance_by_timestamp(self, token_address: str, timestamp: int) -> float:
        """Returns token balance for specified timestamp."""
        if token_address not in self.tokens_data:
            self.tokens_data[token_address] = self._get_user_transactions(token_address, self.user)
        return self.tokens_data[token_address][self.tokens_data[token_address]['timeStamp'] <= timestamp]['balance'].sum()


In [27]:
storage = TokenBalanceStorage(api_key=ETHERSCAN_API_TOKEN, user='0xC99a75F23C50eAAB2595FF19Aa19caAB94a2659e')

In [29]:
storage.get_token_balance_by_timestamp('0xaf88d065e77c8cC2239327C5EDb3A432268e5831', 1700364901-1)

48.955677

In [30]:
storage.tokens_data['0xaf88d065e77c8cC2239327C5EDb3A432268e5831']

,blockNumber,from,to,balance,timeStamp
1,151844050,0xc99a75f23c50eaab2595ff19aa19caab94a2659e,0xd5fa4a76eb66daa09be734ce2ad5f70039b60dd3,-48.950000,1700364901
0,151841770,0x6f2b6796286b83ca46d4c4f19a3fae4f58ae11d1,0xc99a75f23c50eaab2595ff19aa19caab94a2659e,48.955677,1700364249


In [32]:
query =\
"""
{
  liquidates(orderBy: blockNumber) {
    amountUSD
    liquidatee {
      id
    }
    hash
  }
}
"""

url = 'https://gateway.thegraph.com/api/api-key/subgraphs/id/84CvqQHYhydZzr2KSth8s1AFYpBRzUbVJXq6PWuZm9U9'

response = requests.post(url, json={'query': query})
data = json.loads(response.text)
data

{'errors': [{'locations': [{'column': 3, 'line': 3}],
   'message': 'Type `Query` has no field `liquidates`'}]}